A sentiment classifier that outperforms the current top HuggingFace leaderboard on the `rotten_tomatoes` sentiment dataset.

In [1]:
import sys
import os

notebook_dir = os.path.abspath('')
parent_dir = os.path.dirname(notebook_dir)
sys.path.insert(0, parent_dir)

In [2]:
# Auto reload modules
%load_ext autoreload
%autoreload 2

In [3]:
from dataset import Dataset, Embedder
from model import ClassificationModel
import json

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [4]:
# Load the config file
# type = "sentiment_imdb"
type = "sentiment_rottentomatoes"

with open (f'configs/{type}.json') as f:
    config = json.load(f)

In [6]:
# Create the test and validation datasets
embedder = Embedder('E5')
config_dataset = config['datasets'][0]
dataset_train = Dataset(type, config_dataset, split='train')
dataset_test = Dataset(type, config_dataset, split='test')

In [ ]:
# Fit and test the linear model
config_model = config['model']
model = ClassificationModel(config_model)

result_train = model.fit(dataset_train)
print('Training Results', result_train)

result_test = model.test(dataset_test)
print('Validation Results:', result_test)

In [ ]:
path = f'models/{type}/model.npz'
model.save(path)